In [1]:
def conversion(url_larga: str) -> str:
    # Code here

    url_corta = 'https://bit.ly/36Yvv9W'
    return url_corta

def add_liga_publica(url_larga, categoria) -> bool:

    llaveDic=f'{url_larga}'

    # Hacer la conversion liga publica 
    url_corta = conversion(url_larga)

    exito=r.hmset(llaveDic,{'url_corta':url_corta,'categoria':categoria})
    # Agregar a redis
    return exito

def dictBytes_a_dictString(diccionario):
  return {k.decode('utf-8'):v.decode('utf-8') for k,v in diccionario.items()}



# Crear ligas de user

CRUD
Create
Read
Update
Delete

In [2]:
def agregarUser(username,nombre,password):
    exito=r.hmset(username,{'nombre':nombre,'password':password})
    return exito 

def add_liga_publica_user(username, url_larga,categoria) -> True:    
   # Agregar liga publicas en una estructura de datos 
   # de conjuntos

    # checar si existe user, no se necesita


    # También debe agregar a la lista pública
    add_liga_publica(url_larga,categoria)
    
    key_pub_usuario=f'{username}_pub'

    exito = r.sadd(key_pub_usuario,url_larga)

    return exito

def add_liga_privada_user(username, url_larga, categoria) -> True:
    # Crear la estructura de liga privada 
    # y agregar url_larga,url_corta y categoria

    # Creamos el set de ligas privadas
    
    r.sadd(f'lpriv_{username}',url_larga)

    # Creamos un diccionario por cada url_larga
    url_corta=conversion(url_larga)
    
    exito = r.hmset(f'{username}_{url_larga}',{'url_corta':url_corta,'categoria':categoria})
    return exito



In [3]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379)
# r.flushdb()

agregarUser('hugol','hugo','1234')
print(add_liga_privada_user('hugol','youtube.com','musica'))
print(add_liga_privada_user('hugol','youtube2.com','musica'))
print(add_liga_privada_user('hugol','youtube1.com','educacion'))
# Conjuntos
print(add_liga_publica_user('hugol','reddit.com','social'))
print(add_liga_publica_user('hugol','reddit.com','social')) # Regresa 0 porque ya existe

True
True
True
0
0


C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  
C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:32: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  


In [4]:
# Read
existe = r.hmget('hugol', 'nombre')
existe[0] 
r.hmget('hugol', 'password')
#dictBytes_a_dictString()
# [si.decode('utf-8') for si in existe]
# existe[0].decode('utf-8')

[b'1234']

In [5]:
not not " "

True

In [6]:
 existe = r.hmget('hugol', '1324')
 not not existe[0]

False

# Leer ligas user

In [7]:
# Read

def recuperaListas(username):
  cjto_url_pri=r.smembers(f'lpriv_{username}')
  lista_url_pri=[si.decode('utf-8') for si in cjto_url_pri]

  cjto_url_pub=r.smembers(f'{username}_pub')
  lista_url_pub=[si.decode('utf-8') for si in cjto_url_pub]

  categorias_pri={}
  categorias_pub={}
  for url in lista_url_pri:
    llaveDic=f'{username}_{url}'
    dict_url=dictBytes_a_dictString(r.hgetall(llaveDic))
    
    cat=dict_url['categoria']
    url_corta=dict_url['url_corta']

    if not categorias_pri.get(cat):
      categorias_pri[cat]=[url_corta]
    else:
      categorias_pri[cat].append(url_corta)

  for url in lista_url_pub:
    llaveDic=f'{url}'
    dict_url=dictBytes_a_dictString(r.hgetall(llaveDic))
    
    cat=dict_url['categoria']
    url_corta=dict_url['url_corta']

    if not categorias_pub.get(cat):
      categorias_pub[cat]=[url_corta]
    else:
      categorias_pub[cat].append(url_corta)
  

  return categorias_pri,categorias_pub
  
# def recuperaListasPublicas(username):
  
#   return 

recuperaListas('hugol')

({'musica': ['https://bit.ly/36Yvv9W', 'https://bit.ly/36Yvv9W'],
  'educacion': ['https://bit.ly/36Yvv9W']},
 {'social': ['https://bit.ly/36Yvv9W']})

# Borrar ligas de user

In [107]:
# Reseteamos la BD para hacer pruebas
r.flushdb()

# Agregamos registros
agregarUser('hugol','hugo','1234')
add_liga_privada_user('hugol','youtube.com','musica')
add_liga_privada_user('hugol','youtube2.com','musica')
add_liga_privada_user('hugol','youtube1.com','educacion')

add_liga_publica_user('hugol','reddit.com','social')
add_liga_publica_user('hugol','reddit1.com','social')


# Delete 
def borrar_liga_pub(username,url_corta):
  
  for url_larga in r.smembers(f'lpriv_{username}'):
    dicc_nombre=f"{username}_{url_larga.decode('utf-8')}"
    url_corta1=r.hget(dicc_nombre,'url_corta')
    if url_corta==url_corta1.decode('utf-8'):
      # Quitar de conjunto lpriv_hugo
      r.srem(f'lpriv_{username}',url_larga)
      # Quitar diccionario con llave dicc_nombre
      r.delete(dicc_nombre)
      break # Termina el loop si ya se encontró 
  return url_larga

def borrar_liga_priv(username,url_corta):

  for url_larga in r.smembers(f'{username}_pub'):
    dicc_nombre=f"{url_larga.decode('utf-8')}"
    url_corta1=r.hget(dicc_nombre,'url_corta')

    if url_corta==url_corta1.decode('utf-8'):
      # print('Borrar pub:',u2)
      # Quitar de conjunto {username}_pub
      r.srem(f'{username}_pub',url_larga)
      # Quitar diccionario con llave dicc_nombre
      r.delete(dicc_nombre)
      break

  return url_larga

def borrar_liga(username,url_corta,publico=True):
  if publico:
    return borrar_liga_pub(username,url_corta)

  return borrar_liga_priv(username,url_corta)

# Hacemos pruebas
username='hugol'
print(recuperaListas(f'{username}'))
u1='https://bit.ly/TeTI'
print(borrar_liga(username,u1,publico=True))
# print(recuperaListas(f'{username}'))
u2='https://bit.ly/y7Dw'
print(borrar_liga(username,u2,publico=False))
print(recuperaListas('hugol'))

hugol_youtube.com


[b'hugol_youtube1.com', b'hugol', b'lpriv_hugol']

# Actualizar ligas user

In [43]:
# Update

def actualizarLiga(username,liga,categoria_nueva):
  # Si solo se cambia de categoría se deja la entrada
  # if categoria_nueva=='publica':
  #   borrarLiga(username, liga)
  #   add_liga_publica_user(username,liga_nueva)
  
  # elif not liga_nueva:
  
  # # Primero se borra 
  # borrarLiga(username,liga_vieja)
  # # Luego se crea un elemento nuevo
  # add_liga_privada_user(username,url_larga=liga_nueva,categoria=categoria_nueva)
  key_liga=f'{username}_{liga}'
  exito=r.hset(name=key_liga,key='categoria',value=categoria_nueva)

  return exito

actualizarLiga('hugol','youtube1.com','wishlist')

0

In [44]:
recuperaListas('hugol')

{'wishlist': ['https://bit.ly/36Yvv9W'],
 'musica': ['https://bit.ly/36Yvv9W', 'https://bit.ly/36Yvv9W'],
 'social': ['https://bit.ly/36Yvv9W']}